In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
df_basa_full=pd.read_csv('basa_original.csv')

In [4]:
non_aquatic_species = 'Abies,Pinus,Juniperus,Taxus,Betula,Corylus,Alnus,Carpinus,Salix,Ulmus,Populus,Acer,Fraxinus,Fagus,Tilia,Juglans,Castanea,Quercus caducifolio,Quercus perennifolio,Pistacia,Rhamnus,Phillyrea,Buxus,Sambucus,Viburnum,Sanguisorba,Tamarix,Thymelaeaceae,Ephedra distachya,Ephedra fragilis,Ericaceae,Hereda helix,Ilex aquifolium,Viscum album,Lonicera,Vitis,Oleaceae,Myrtus,Olea,Poaceae,Lygeum spartum,Artemisia,Cichorioideae,Asteroideae,Cardueae,Rubiaceae,Centaurea,Chenopodiaceae,Caryophyllaceae,Plantago,Brassicaceae,Saxifragaceae,Fabaceae,Genista,Lotus type,Trifolium type,Rosaceae,Ribes,Boraginaceae,Sedum,Helianthemum,Lamiaceae,Urticaceae,Rumex,Berberidaceae,Euphorbiaceae,Primulaceae,Scrophulariaceae,Papaver,Campanulaceae,Convolvulaceae,Liliaceae,Iridaceae,Crassulaceae,Ranunculaceae,Cistaceae,Galium,Apiaceae,Valerianaceae,Cerealia type,Polygonaceae,Ranunculus'

species_mapping = {
        "Dec_Querc": "Quercus caducifolio",
        "Ever_Querc": "Quercus perennifolio",
        "Ephedra dist": "Ephedra distachya",
        "Ephedra frag": "Ephedra fragilis",
        "Lygeum": "Lygeum spartum",
        "Cicho": "Cichorioideae",
        "Astroi": "Asteroideae",
        "Carduaceae": "Cardueae",
        "Rubiac": "Rubiaceae",
        "Chenopo": "Chenopodiaceae",
        "Caryphy": "Caryophyllaceae",
        "Brassicac": "Brassicaceae",
        "Saxifrag": "Saxifragaceae",
        "Boraginac": "Boraginaceae",
        "Helianthem": "Helianthemum",
        "Euphorbiac": "Euphorbiaceae",
        "Primulac": "Primulaceae",
        "Scrophulari": "Scrophulariaceae",
        "Campanulac": "Campanulaceae",
        "Valerian": "Valerianaceae",
        "Cerealia": "Cerealia type",
        "Polygon": "Polygonaceae"
    }

In [5]:
non_aquatic_species = [s.strip() for s in non_aquatic_species.split(',')]
df_bas = df_basa_full[
    list(df_basa_full.columns[:8]) +
    [c for c in df_basa_full.columns[8:] if c in non_aquatic_species]
]

df_bas.head()

,depth,cal BP,density,accrate,weight,volume,lycadd,lyc,Abies,Pinus,...,Liliaceae,Iridaceae,Crassulaceae,Ranunculaceae,Cistaceae,Galium,Apiaceae,Valerianaceae,Polygonaceae,Ranunculus
0,0.5,-56.90,1.5,0.46,4.3,2.866667,24200,26,1,189,...,2,2,1,2,1,0,0,0,0,1
1,5.5,-50.33,1.7,0.46,3.4,2.000000,24200,12,1,149,...,0,0,0,5,0,0,1,0,0,3
2,10.5,-42.11,1.5,0.46,3.2,2.133333,24200,58,0,206,...,1,1,1,6,0,1,2,0,0,5
3,15.5,-30.47,1.7,0.46,4.0,2.352941,24200,29,0,252,...,0,0,0,4,0,1,0,0,0,2
4,20.5,-13.25,1.7,0.09,3.2,1.882353,24200,17,0,141,...,0,0,0,4,0,0,3,0,1,3


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pygam import LinearGAM, s
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.api import VAR
import warnings
warnings.filterwarnings('ignore')

# ============================================
# CONFIGURACIÓN
# ============================================

especies_cols = df_bas.columns[df_bas.columns.get_loc('Abies'):].tolist()
print(f"Total especies: {len(especies_cols)}")

# PARÁMETROS
UMBRAL_P_VALUE = 0.01
MIN_PRESENCIA = 3

# GAM
N_SPLINES_BASE = 140
LAMBDA_GAM = 0.01
N_PUNTOS_EQUIDISTANTES = 140  # Mismo número de puntos

print(f"\nParámetros:")
print(f"  Umbral p-value: {UMBRAL_P_VALUE}")
print(f"  Mínimo puntos presencia: {MIN_PRESENCIA}")
print(f"  Splines base: {N_SPLINES_BASE}")
print(f"  Lambda: {LAMBDA_GAM}")
print(f"  Puntos equidistantes: {N_PUNTOS_EQUIDISTANTES}")
print("="*60)

# ============================================
# FUNCIONES
# ============================================

def es_serie_valida(serie):
    """Verifica si una serie es válida para análisis"""
    n_presencia = (serie > 0).sum()
    if n_presencia < MIN_PRESENCIA:
        return False
    if serie.nunique() < 2:
        return False
    return True

def reconstruir_gam_equidistante(df_original, n_puntos=140):
    """
    Reconstruye con GAM y genera tiempo EQUIDISTANTE
    """
    tiempo_original = df_original['cal BP'].values
    
    # Crear tiempo equidistante (de antiguo a reciente)
    tiempo_equidistante = np.linspace(tiempo_original.min(), tiempo_original.max(), n_puntos)
    
    df_reconstruido = pd.DataFrame({'cal BP': tiempo_equidistante})
    
    print("\n=== RECONSTRUYENDO CON GAM (TIEMPO EQUIDISTANTE) ===")
    
    especies_procesadas = 0
    
    for idx, sp in enumerate(especies_cols):
        if idx % 10 == 0 and idx > 0:
            print(f"  Procesadas {idx}/{len(especies_cols)} especies")
            
        serie = df_original[sp].copy()
        
        if not es_serie_valida(serie):
            continue
        
        # Puntos originales (con tiempo original)
        tiempo_valido = tiempo_original.reshape(-1, 1)
        valores = serie.values
        
        try:
            # GAM
            gam = LinearGAM(s(0, n_splines=N_SPLINES_BASE, lam=LAMBDA_GAM))
            gam.fit(tiempo_valido, valores)
            
            # Predecir en tiempo EQUIDISTANTE
            tiempo_pred = tiempo_equidistante.reshape(-1, 1)
            predicciones = gam.predict(tiempo_pred)
            predicciones = np.maximum(predicciones, 0)
            
            df_reconstruido[sp] = predicciones
            especies_procesadas += 1
            
        except Exception as e:
            # Si falla, interpolar linealmente en tiempo equidistante
            from scipy.interpolate import interp1d
            try:
                f = interp1d(tiempo_original, valores, kind='linear', 
                            fill_value='extrapolate', bounds_error=False)
                predicciones = f(tiempo_equidistante)
                predicciones = np.maximum(predicciones, 0)
                df_reconstruido[sp] = predicciones
            except:
                df_reconstruido[sp] = np.mean(valores)
    
    print(f"\n✅ Especies procesadas: {especies_procesadas}")
    
    # Ordenar de antiguo a reciente
    df_reconstruido = df_reconstruido.sort_values('cal BP', ascending=True).reset_index(drop=True)
    
    return df_reconstruido

def definir_ventanas_equidistantes(df_reconstruido):
    """
    Define ventanas en el tiempo EQUIDISTANTE basado en los valores de cal BP
    """
    cal_values = df_reconstruido['cal BP'].values
    
    # Rangos deseados
    rangos = [
        ("9798_6253", 9798, 6253),
        ("6182_3842", 6182, 3842),
        ("3771_-57", 3771, -57)
    ]
    
    windows = {}
    for name, cal_max, cal_min in rangos:
        # Encontrar índices que corresponden a estos cal BP
        mask = (cal_values <= cal_max) & (cal_values >= cal_min)
        indices = np.where(mask)[0]
        
        if len(indices) > 0:
            windows[name] = (indices[0], indices[-1])
            print(f"  {name}: cal BP {cal_values[indices[0]]:.0f} - {cal_values[indices[-1]]:.0f} "
                  f"(filas {indices[0]}-{indices[-1]})")
    
    return windows

def calcular_granger(df_window, especies_validas):
    """Calcula causalidad de Granger"""
    resultados = []
    
    for i, sp1 in enumerate(especies_validas):
        for j, sp2 in enumerate(especies_validas):
            if i != j:
                try:
                    array_x = df_window[sp1].values[::-1]
                    array_y = df_window[sp2].values[::-1]
                    
                    # Ruido adaptativo
                    ruido_x = np.random.normal(0, np.std(array_x) * 1e-6, len(array_x))
                    ruido_y = np.random.normal(0, np.std(array_y) * 1e-6, len(array_y))
                    
                    array_x = array_x + ruido_x
                    array_y = array_y + ruido_y
                    
                    # VAR
                    data = pd.DataFrame({sp1: array_x, sp2: array_y})
                    scaler = StandardScaler()
                    data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
                    
                    model = VAR(data_scaled)
                    results = model.fit(maxlags=1)
                    
                    causality = results.test_causality(caused=sp2, causing=[sp1], kind='f')
                    p_value = causality.pvalue
                    coeficiente = results.coefs[0][1][0]
                    
                    resultados.append({
                        'from': sp1,
                        'to': sp2,
                        'p_value': p_value,
                        'coeficiente': coeficiente,
                        'signo': 'positivo' if coeficiente > 0 else 'negativo'
                    })
                    
                except:
                    continue
    
    return pd.DataFrame(resultados)

# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================

# 1. Reconstruir con GAM en tiempo EQUIDISTANTE
df_equidistante = reconstruir_gam_equidistante(df_bas, N_PUNTOS_EQUIDISTANTES)

print(f"\nDatos reconstruidos: {len(df_equidistante)} puntos")
print(f"Rango cal BP: {df_equidistante['cal BP'].min():.0f} - {df_equidistante['cal BP'].max():.0f}")

# 2. Definir ventanas en el NUEVO tiempo equidistante
print("\n=== DEFINICIÓN DE VENTANAS (tiempo equidistante) ===")
windows_equidistantes = definir_ventanas_equidistantes(df_equidistante)

# 3. Identificar especies válidas (usando datos originales)
especies_validas = []
for sp in especies_cols:
    if es_serie_valida(df_bas[sp]):
        especies_validas.append(sp)

print(f"\nEspecies válidas para Granger: {len(especies_validas)}")

# 4. Calcular Granger en cada ventana
resultados = {}

for window_name, (start, end) in windows_equidistantes.items():
    print(f"\n{'='*50}")
    print(f"VENTANA: {window_name}")
    print(f"{'='*50}")
    
    df_window = df_equidistante.iloc[start:end+1].copy()
    print(f"  Filas {start}-{end} ({len(df_window)} puntos)")
    print(f"  Rango cal BP: {df_window['cal BP'].min():.0f} - {df_window['cal BP'].max():.0f}")
    
    df_res = calcular_granger(df_window, especies_validas)
    resultados[window_name] = df_res
    
    if len(df_res) > 0:
        sig = df_res[df_res['p_value'] < UMBRAL_P_VALUE]
        n_sig = len(sig)
        print(f"  Pares analizados: {len(df_res)}")
        print(f"  Links significativos (p<{UMBRAL_P_VALUE}): {n_sig} ({n_sig/len(df_res)*100:.2f}%)")
        
        if n_sig > 0:
            n_pos = len(sig[sig['signo'] == 'positivo'])
            n_neg = len(sig[sig['signo'] == 'negativo'])
            print(f"    Positivos: {n_pos} ({n_pos/n_sig*100:.1f}%)")
            print(f"    Negativos: {n_neg} ({n_neg/n_sig*100:.1f}%)")

# ============================================
# COMPARACIÓN CON RESULTADOS ESPERADOS
# ============================================
print("\n" + "="*70)
print("COMPARACIÓN CON RESULTADOS DEL PAPER")
print("="*70)

esperados = {
    "9798_6253": {"links": 161, "pct_neg": 9.9},
    "6182_3842": {"links": 252, "pct_neg": 29.8},
    "3771_-57": {"links": 155, "pct_neg": 22.6}
}

print("\nVentana        | Obtenido       | Esperado       | Diferencia")
print("-"*70)

for window_name in windows_equidistantes.keys():
    if window_name in resultados and len(resultados[window_name]) > 0:
        df_res = resultados[window_name]
        sig = df_res[df_res['p_value'] < UMBRAL_P_VALUE]
        n_sig = len(sig)
        
        if n_sig > 0:
            n_neg = len(sig[sig['signo'] == 'negativo'])
            pct_neg = n_neg / n_sig * 100
        else:
            n_sig = 0
            pct_neg = 0
        
        exp = esperados.get(window_name, {"links": 0, "pct_neg": 0})
        
        diff_links = n_sig - exp["links"]
        diff_pct = pct_neg - exp["pct_neg"]
        
        print(f"{window_name:12} | {n_sig:3} links ({pct_neg:4.1f}%) | "
              f"{exp['links']:3} links ({exp['pct_neg']:4.1f}%) | "
              f"links: {diff_links:+d}, %: {diff_pct:+.1f}")
    else:
        print(f"{window_name:12} | Sin datos")

Total especies: 79

Parámetros:
  Umbral p-value: 0.01
  Mínimo puntos presencia: 3
  Splines base: 140
  Lambda: 0.01
  Puntos equidistantes: 140

=== RECONSTRUYENDO CON GAM (TIEMPO EQUIDISTANTE) ===
  Procesadas 10/79 especies
  Procesadas 20/79 especies
  Procesadas 30/79 especies
  Procesadas 40/79 especies
  Procesadas 50/79 especies
  Procesadas 60/79 especies
  Procesadas 70/79 especies

✅ Especies procesadas: 75

Datos reconstruidos: 140 puntos
Rango cal BP: -57 - 9798

=== DEFINICIÓN DE VENTANAS (tiempo equidistante) ===
  9798_6253: cal BP 6253 - 9798 (filas 89-139)
  6182_3842: cal BP 3843 - 6111 (filas 55-87)
  3771_-57: cal BP -57 - 3701 (filas 0-53)

Especies válidas para Granger: 75

VENTANA: 9798_6253
  Filas 89-139 (51 puntos)
  Rango cal BP: 6253 - 9798
  Pares analizados: 5550
  Links significativos (p<0.01): 201 (3.62%)
    Positivos: 184 (91.5%)
    Negativos: 17 (8.5%)

VENTANA: 6182_3842
  Filas 55-87 (33 puntos)
  Rango cal BP: 3843 - 6111
  Pares analizados: 55

In [19]:
# ============================================
# GUARDAR DATOS EQUIDISTANTES TRAS GAM
# ============================================

import pickle

# Después de obtener df_equidistante de la función reconstruir_gam_equidistante()
# df_equidistante = reconstruir_gam_equidistante(df_bas, N_PUNTOS_EQUIDISTANTES)

print("\n" + "="*60)
print("GUARDANDO DATOS EQUIDISTANTES")
print("="*60)

print(f"DataFrame equidistante:")
print(f"  Forma: {df_equidistante.shape}")
print(f"  Columnas: {df_equidistante.columns.tolist()}")
print(f"  Rango cal BP: {df_equidistante['cal BP'].min():.0f} - {df_equidistante['cal BP'].max():.0f}")
print(f"  Número de puntos: {len(df_equidistante)}")

# ============================================
# OPCIÓN 1: Guardar con pickle (formato binario nativo de Python)
# ============================================

with open('pkl/gam_original.pkl', 'wb') as f:
    pickle.dump(df_equidistante, f)

print("\n✅ Datos guardados en 'gam_original.pkl' (formato pickle)")

# ============================================
# VERIFICAR QUE SE GUARDÓ CORRECTAMENTE
# ============================================

print("\n" + "="*60)
print("VERIFICANDO CARGA DE DATOS")
print("="*60)

# Cargar con pickle
with open('pkl/gam_original.pkl', 'rb') as f:
    df_cargado_pickle = pickle.load(f)

print(f"Pickle - cargado: {df_cargado_pickle.shape}, "
      f"cal BP: {df_cargado_pickle['cal BP'].min():.0f} - {df_cargado_pickle['cal BP'].max():.0f}")



# ============================================
# INFORMACIÓN PARA FUTUROS ANÁLISIS
# ============================================

print("\n" + "="*60)
print("METADATOS")
print("="*60)

metadata = {
    'fecha_generacion': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'n_puntos_original': len(df_bas),
    'n_puntos_equidistante': len(df_equidistante),
    'n_especies': len(especies_cols),
    'rango_cal_BP': f"{df_equidistante['cal BP'].min():.0f} - {df_equidistante['cal BP'].max():.0f}",
    'parametros_gam': {
        'n_splines': N_SPLINES_BASE,
        'lambda': LAMBDA_GAM,
        'min_presencia': MIN_PRESENCIA
    }
}

print("\nMetadatos del archivo:")
for key, value in metadata.items():
    if key != 'parametros_gam':
        print(f"  {key}: {value}")
    else:
        print(f"  {key}:")
        for k, v in value.items():
            print(f"    {k}: {v}")

# Guardar metadatos en archivo de texto
with open('metadatos_equidistantes.txt', 'w') as f:
    f.write("METADATOS - DATOS EQUIDISTANTES GAM\n")
    f.write("="*40 + "\n")
    for key, value in metadata.items():
        f.write(f"{key}: {value}\n")

print("\n✅ Metadatos guardados en 'metadatos_equidistantes.txt'")


GUARDANDO DATOS EQUIDISTANTES
DataFrame equidistante:
  Forma: (140, 76)
  Columnas: ['cal BP', 'Abies', 'Pinus', 'Juniperus', 'Taxus', 'Betula', 'Corylus', 'Alnus', 'Carpinus', 'Salix', 'Ulmus', 'Populus', 'Acer', 'Fraxinus', 'Fagus', 'Tilia', 'Juglans', 'Castanea', 'Pistacia', 'Rhamnus', 'Buxus', 'Sambucus', 'Viburnum', 'Tamarix', 'Thymelaeaceae', 'Ephedra distachya', 'Ephedra fragilis', 'Ericaceae', 'Hereda helix', 'Ilex aquifolium', 'Lonicera', 'Vitis', 'Oleaceae', 'Myrtus', 'Olea', 'Poaceae', 'Artemisia', 'Cichorioideae', 'Asteroideae', 'Cardueae', 'Rubiaceae', 'Centaurea', 'Chenopodiaceae', 'Caryophyllaceae', 'Plantago', 'Brassicaceae', 'Saxifragaceae', 'Fabaceae', 'Genista', 'Lotus type', 'Trifolium type', 'Rosaceae', 'Ribes', 'Boraginaceae', 'Sedum', 'Helianthemum', 'Lamiaceae', 'Urticaceae', 'Rumex', 'Berberidaceae', 'Euphorbiaceae', 'Primulaceae', 'Scrophulariaceae', 'Papaver', 'Campanulaceae', 'Convolvulaceae', 'Liliaceae', 'Iridaceae', 'Crassulaceae', 'Ranunculaceae', 'Cis

In [9]:
import numpy as np
import pandas as pd
import pickle
import os
from scipy import stats

# ============================================
# CONFIGURACIÓN DEL BOOTSTRAP ESTACIONARIO
# ============================================

# Crear carpeta para guardar los archivos
os.makedirs('pkl', exist_ok=True)

L = 140  # Longitud de la serie temporal
N_BOOTSTRAP = 5  # Número de matrices bootstrap
lambda_exp = 1/L  # Parámetro para la distribución exponencial

print("="*60)
print("BOOTSTRAP ESTACIONARIO")
print("="*60)
print(f"Longitud de la serie (L): {L}")
print(f"Número de bootstraps: {N_BOOTSTRAP}")
print(f"Lambda distribución exponencial: {lambda_exp:.4f}")
print("="*60)

# ============================================
# FUNCIÓN PARA GENERAR UNA SERIE BOOTSTRAP
# ============================================

def generar_serie_bootstrap(serie_original, L, lambda_exp):
    """
    Genera una serie bootstrap usando el método de bootstrap estacionario
    
    Parámetros:
    - serie_original: array de longitud L con la serie temporal original
    - L: longitud de la serie
    - lambda_exp: parámetro para la distribución exponencial
    
    Retorna:
    - serie_bootstrap: array de longitud L con la serie bootstrap
    """
    
    serie_bootstrap = np.zeros(L)
    pos = 0
    
    while pos < L:
        # 1. Elegir m (punto de inicio) uniformemente
        m = np.random.randint(0, L)
        
        # 2. Elegir l (longitud del bloque) con distribución exponencial
        l = int(np.random.exponential(scale=1/lambda_exp))
        l = min(l, L)  # Asegurar que no excede L
        
        # 3. Copiar el bloque de longitud l desde m
        for i in range(l):
            if pos >= L:
                break
                
            # Calcular índice en la serie original (con wrap-around)
            idx_original = (m + i) % L
            serie_bootstrap[pos] = serie_original[idx_original]
            pos += 1
    
    return serie_bootstrap

def generar_bootstrap_completo(df_equidistante, especies_cols, L, lambda_exp):
    """
    Genera un DataFrame bootstrap completo con todas las especies
    """
    # Crear DataFrame vacío con la misma estructura
    df_bootstrap = pd.DataFrame()
    df_bootstrap['cal BP'] = df_equidistante['cal BP'].values  # Mantener el tiempo
    
    # Para cada especie, generar su serie bootstrap
    for sp in especies_cols:
        if sp in df_equidistante.columns:
            serie_original = df_equidistante[sp].values
            serie_bootstrap = generar_serie_bootstrap(serie_original, L, lambda_exp)
            df_bootstrap[sp] = serie_bootstrap
    
    return df_bootstrap

# ============================================
# GENERAR Y GUARDAR LOS 5 BOOTSTRAPS
# ============================================

# Asegurarnos de que df_equidistante existe
# Si no, cargarlo (asumiendo que está en el entorno)
if 'df_equidistante' not in locals() and 'df_equidistante' not in globals():
    print("\nCargando datos equidistantes...")
    try:
        with open('datos_equidistantes_gam.pkl', 'rb') as f:
            df_equidistante = pickle.load(f)
        print("✅ Datos cargados correctamente")
    except FileNotFoundError:
        print("❌ Error: No se encuentra el archivo 'datos_equidistantes_gam.pkl'")
        print("   Asegúrate de haber ejecutado primero el código de GAM")
        exit()

# Verificar que tenemos las columnas de especies
especies_cols_bootstrap = [col for col in especies_cols if col in df_equidistante.columns]
print(f"\nEspecies disponibles para bootstrap: {len(especies_cols_bootstrap)}")

# Fijar semilla para reproducibilidad (opcional)
np.random.seed(42)

# Generar y guardar cada bootstrap
for i in range(1, N_BOOTSTRAP + 1):
    print(f"\n{'='*50}")
    print(f"GENERANDO BOOTSTRAP #{i}")
    print(f"{'='*50}")
    
    # Generar bootstrap
    df_bootstrap = generar_bootstrap_completo(df_equidistante, especies_cols_bootstrap, L, lambda_exp)
    
    # Verificar dimensiones
    print(f"  Dimensiones: {df_bootstrap.shape}")
    print(f"  Columnas: {df_bootstrap.columns.tolist()[:5]}...")
    
    # Guardar en archivo pickle
    filename = f'pkl/gam_boost_{i}.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(df_bootstrap, f)
    
    print(f"  ✅ Guardado en: {filename}")
    
    # Verificación rápida (mostrar primeros valores de algunas especies)
    print(f"\n  Primeros valores de especies ejemplo:")
    especies_ejemplo = df_bootstrap.columns[1:6].tolist()
    for sp in especies_ejemplo:
        if sp in df_bootstrap.columns:
            print(f"    {sp}: {df_bootstrap[sp].values[:5]}")

# ============================================
# VERIFICACIÓN FINAL
# ============================================

print("\n" + "="*60)
print("VERIFICACIÓN DE ARCHIVOS GENERADOS")
print("="*60)

archivos_generados = []
for i in range(1, N_BOOTSTRAP + 1):
    filename = f'pkl/gam_boost_{i}.pkl'
    if os.path.exists(filename):
        size_kb = os.path.getsize(filename) / 1024
        archivos_generados.append((filename, size_kb))

print(f"\nArchivos encontrados en carpeta 'pkl/':")
for filename, size in archivos_generados:
    print(f"  📄 {filename} ({size:.1f} KB)")

if len(archivos_generados) == N_BOOTSTRAP:
    print(f"\n✅ ÉXITO: Se generaron los {N_BOOTSTRAP} archivos bootstrap")
else:
    print(f"\n⚠️ ERROR: Solo se generaron {len(archivos_generados)} de {N_BOOTSTRAP} archivos")

# ============================================
# FUNCIÓN PARA CARGAR UN BOOTSTRAP ESPECÍFICO
# ============================================

def cargar_bootstrap(numero):
    """
    Carga un archivo bootstrap específico
    """
    filename = f'pkl/gam_boost_{numero}.pkl'
    try:
        with open(filename, 'rb') as f:
            df = pickle.load(f)
        print(f"✅ Cargado bootstrap #{numero}")
        return df
    except FileNotFoundError:
        print(f"❌ No se encuentra el archivo: {filename}")
        return None

# Ejemplo de uso
print("\n" + "="*60)
print("EJEMPLO DE CARGA")
print("="*60)

# Cargar el primer bootstrap como ejemplo
df_ejemplo = cargar_bootstrap(1)
if df_ejemplo is not None:
    print(f"\n  Dimensiones: {df_ejemplo.shape}")
    print(f"  Rango cal BP: {df_ejemplo['cal BP'].min():.0f} - {df_ejemplo['cal BP'].max():.0f}")
    print(f"\n  Primeras 3 filas:")
    print(df_ejemplo[['cal BP'] + especies_ejemplo[:3]].head(3).to_string())

print("\n" + "="*60)
print("✅ PROCESO COMPLETADO")
print("="*60)

BOOTSTRAP ESTACIONARIO
Longitud de la serie (L): 140
Número de bootstraps: 5
Lambda distribución exponencial: 0.0071

Especies disponibles para bootstrap: 75

GENERANDO BOOTSTRAP #1
  Dimensiones: (140, 76)
  Columnas: ['cal BP', 'Abies', 'Pinus', 'Juniperus', 'Taxus']...
  ✅ Guardado en: pkl/gam_boost_1.pkl

  Primeros valores de especies ejemplo:
    Abies: [0.61271431 0.20884143 0.         0.02506226 0.        ]
    Pinus: [121.52079654 145.23044146 146.71232116  95.49331131 106.9890442 ]
    Juniperus: [4.46750261 3.8380869  3.95351712 3.55912202 4.70149744]
    Taxus: [3.31136103e-04 0.00000000e+00 0.00000000e+00 2.92538639e-10
 2.50699598e-09]
    Betula: [1.27731334 6.84360917 9.25204574 4.65463797 2.08368075]

GENERANDO BOOTSTRAP #2
  Dimensiones: (140, 76)
  Columnas: ['cal BP', 'Abies', 'Pinus', 'Juniperus', 'Taxus']...
  ✅ Guardado en: pkl/gam_boost_2.pkl

  Primeros valores de especies ejemplo:
    Abies: [0.00000000e+00 1.22188357e-02 1.12886580e-01 0.00000000e+00
 1.59805

In [12]:
# ============================================
# SCRIPT 1: GENERAR BOOTSTRAPS ESTACIONARIOS
# ============================================

import numpy as np
import pandas as pd
import pickle
import os

print("="*60)
print("SCRIPT 1: GENERANDO BOOTSTRAPS ESTACIONARIOS")
print("="*60)

# ============================================
# CONFIGURACIÓN
# ============================================

# Crear carpeta para guardar los archivos
os.makedirs('pkl', exist_ok=True)

L = 140  # Longitud de la serie temporal
N_BOOTSTRAP = 5  # Número de matrices bootstrap (cambiar a 1000 después)
lambda_exp = 1/L  # Parámetro para la distribución exponencial

# Cargar datos originales equidistantes
print("\nCargando datos equidistantes...")
with open('pkl/gam_original.pkl', 'rb') as f:
    df_original = pickle.load(f)

# Identificar especies (excluyendo 'cal BP')
especies_cols = [col for col in df_original.columns if col != 'cal BP']
print(f"Especies disponibles: {len(especies_cols)}")

print(f"\nParámetros:")
print(f"  Longitud serie (L): {L}")
print(f"  Número bootstraps: {N_BOOTSTRAP}")
print(f"  Lambda exponencial: {lambda_exp:.4f}")

# ============================================
# FUNCIÓN PARA GENERAR UNA SERIE BOOTSTRAP
# ============================================

def generar_serie_bootstrap(serie_original, L, lambda_exp):
    """
    Genera una serie bootstrap usando el método de bootstrap estacionario
    
    Parámetros:
    - serie_original: array de longitud L con la serie temporal original
    - L: longitud de la serie
    - lambda_exp: parámetro para la distribución exponencial
    
    Retorna:
    - serie_bootstrap: array de longitud L con la serie bootstrap
    """
    
    serie_bootstrap = np.zeros(L)
    pos = 0
    
    while pos < L:
        # 1. Elegir m (punto de inicio) uniformemente
        m = np.random.randint(0, L)
        
        # 2. Elegir l (longitud del bloque) con distribución exponencial
        l = int(np.random.exponential(scale=1/lambda_exp))
        l = min(l, L)  # Asegurar que no excede L
        
        # 3. Copiar el bloque de longitud l desde m
        for i in range(l):
            if pos >= L:
                break
                
            # Calcular índice en la serie original (con wrap-around)
            idx_original = (m + i) % L
            serie_bootstrap[pos] = serie_original[idx_original]
            pos += 1
    
    return serie_bootstrap

def generar_bootstrap_completo(df_original, especies_cols, L, lambda_exp):
    """
    Genera un DataFrame bootstrap completo con todas las especies
    """
    # Crear DataFrame vacío con la misma estructura
    df_bootstrap = pd.DataFrame()
    df_bootstrap['cal BP'] = df_original['cal BP'].values  # Mantener el tiempo
    
    # Para cada especie, generar su serie bootstrap
    for sp in especies_cols:
        if sp in df_original.columns:
            serie_original = df_original[sp].values
            serie_bootstrap = generar_serie_bootstrap(serie_original, L, lambda_exp)
            df_bootstrap[sp] = serie_bootstrap
    
    return df_bootstrap

# ============================================
# GENERAR Y GUARDAR BOOTSTRAPS
# ============================================

print("\n" + "="*60)
print("GENERANDO BOOTSTRAPS")
print("="*60)

# Fijar semilla para reproducibilidad
np.random.seed(42)

# Generar y guardar cada bootstrap
for i in range(1, N_BOOTSTRAP + 1):
    print(f"\n--- Bootstrap #{i} ---")
    
    # Generar bootstrap
    df_bootstrap = generar_bootstrap_completo(df_original, especies_cols, L, lambda_exp)
    
    # Verificar dimensiones
    print(f"  Dimensiones: {df_bootstrap.shape}")
    
    # Guardar en archivo pickle
    filename = f'pkl/gam_boost_{i}.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(df_bootstrap, f)
    
    print(f"  ✅ Guardado en: {filename}")

# ============================================
# VERIFICACIÓN FINAL
# ============================================

print("\n" + "="*60)
print("VERIFICACIÓN DE ARCHIVOS")
print("="*60)

archivos = []
for i in range(1, N_BOOTSTRAP + 1):
    filename = f'pkl/gam_boost_{i}.pkl'
    if os.path.exists(filename):
        size_kb = os.path.getsize(filename) / 1024
        archivos.append((filename, size_kb))

print(f"\nArchivos en carpeta 'pkl/':")
for filename, size in archivos:
    print(f"  📄 {filename} ({size:.1f} KB)")

if len(archivos) == N_BOOTSTRAP:
    print(f"\n✅ ÉXITO: Se generaron los {N_BOOTSTRAP} archivos bootstrap")
else:
    print(f"\n⚠️ ERROR: Solo se generaron {len(archivos)} de {N_BOOTSTRAP} archivos")

print("\n" + "="*60)
print("SCRIPT 1 COMPLETADO")
print("="*60)

SCRIPT 1: GENERANDO BOOTSTRAPS ESTACIONARIOS

Cargando datos equidistantes...
Especies disponibles: 75

Parámetros:
  Longitud serie (L): 140
  Número bootstraps: 5
  Lambda exponencial: 0.0071

GENERANDO BOOTSTRAPS

--- Bootstrap #1 ---
  Dimensiones: (140, 76)
  ✅ Guardado en: pkl/gam_boost_1.pkl

--- Bootstrap #2 ---
  Dimensiones: (140, 76)
  ✅ Guardado en: pkl/gam_boost_2.pkl

--- Bootstrap #3 ---
  Dimensiones: (140, 76)
  ✅ Guardado en: pkl/gam_boost_3.pkl

--- Bootstrap #4 ---
  Dimensiones: (140, 76)
  ✅ Guardado en: pkl/gam_boost_4.pkl

--- Bootstrap #5 ---
  Dimensiones: (140, 76)
  ✅ Guardado en: pkl/gam_boost_5.pkl

VERIFICACIÓN DE ARCHIVOS

Archivos en carpeta 'pkl/':
  📄 pkl/gam_boost_1.pkl (88.7 KB)
  📄 pkl/gam_boost_2.pkl (88.7 KB)
  📄 pkl/gam_boost_3.pkl (88.7 KB)
  📄 pkl/gam_boost_4.pkl (88.7 KB)
  📄 pkl/gam_boost_5.pkl (88.7 KB)

✅ ÉXITO: Se generaron los 5 archivos bootstrap

SCRIPT 1 COMPLETADO


In [ ]:
# ============================================
# SCRIPT 2: ANÁLISIS GRANGER CON BOOTSTRAP POR VENTANAS
# ============================================

import numpy as np
import pandas as pd
import pickle
import os
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SCRIPT 2: ANÁLISIS GRANGER CON BOOTSTRAP POR VENTANAS")
print("="*60)

# ============================================
# CONFIGURACIÓN
# ============================================

# Ventanas temporales (en términos de CAL BP, no índices)
windows = {
    "9798_6253": (9798, 6253),  # Ventana antigua
    "6182_3842": (6182, 3842),  # Ventana media
    "3771_-57": (3771, -57)     # Ventana reciente
}

# Parámetros
UMBRAL_P_VALUE = 0.01  # Para significancia en Granger
N_BOOTSTRAP = 5  # Debe coincidir con el script anterior
ALPHA = 0.05  # Nivel de significancia (2.5% cada cola)

print(f"\nParámetros:")
print(f"  Umbral p-value: {UMBRAL_P_VALUE}")
print(f"  Número bootstraps: {N_BOOTSTRAP}")
print(f"  Alpha (IC 95%): {ALPHA}")
print(f"  Ventanas: {list(windows.keys())}")

# ============================================
# FUNCIONES AUXILIARES
# ============================================

def extraer_ventana(df, cal_min, cal_max):
    """
    Extrae una ventana temporal basada en valores de cal BP
    Maneja correctamente rangos donde cal_min > cal_max
    """
    if cal_min > cal_max:
        # Ventana con límite superior e inferior invertidos
        mask = (df['cal BP'] >= cal_max) & (df['cal BP'] <= cal_min)
    else:
        # Ventana normal
        mask = (df['cal BP'] >= cal_min) & (df['cal BP'] <= cal_max)
    
    df_ventana = df[mask].copy().reset_index(drop=True)
    print(f"    Ventana {cal_max}-{cal_min}: {len(df_ventana)} puntos")
    return df_ventana

def calcular_coeficiente_granger(df, sp1, sp2):
    """
    Calcula el coeficiente de causalidad de Granger sp1 → sp2
    Devuelve el coeficiente o None si hay error
    """
    try:
        # Series
        array_x = df[sp1].values[::-1]
        array_y = df[sp2].values[::-1]
        
        # Verificar que no son constantes
        if np.std(array_x) < 1e-10 or np.std(array_y) < 1e-10:
            return None
        
        # Ruido adaptativo
        ruido_x = np.random.normal(0, np.std(array_x) * 1e-6, len(array_x))
        ruido_y = np.random.normal(0, np.std(array_y) * 1e-6, len(array_y))
        
        array_x = array_x + ruido_x
        array_y = array_y + ruido_y
        
        # VAR
        data = pd.DataFrame({sp1: array_x, sp2: array_y})
        scaler = StandardScaler()
        data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
        
        model = VAR(data_scaled)
        results = model.fit(maxlags=1)
        
        # Coeficiente de sp1 en ecuación de sp2
        coeficiente = results.coefs[0][1][0]
        
        return coeficiente
        
    except Exception as e:
        return None

def calcular_matriz_coeficientes(df, especies_cols, nombre_ventana=""):
    """
    Calcula matriz de coeficientes para todos los pares
    """
    n = len(especies_cols)
    matriz = np.zeros((n, n))
    
    print(f"  Calculando matriz {n}x{n} para {nombre_ventana}...")
    total_pares = n * (n - 1)
    contador = 0
    
    for i, sp1 in enumerate(especies_cols):
        for j, sp2 in enumerate(especies_cols):
            if i != j:
                coef = calcular_coeficiente_granger(df, sp1, sp2)
                if coef is not None:
                    matriz[i, j] = coef
                
                contador += 1
                if contador % 500 == 0:
                    print(f"    Progreso: {contador}/{total_pares}")
    
    return matriz

# ============================================
# PASO 1: CARGAR DATOS
# ============================================

print("\n" + "="*60)
print("PASO 1: CARGANDO DATOS")
print("="*60)

# Cargar datos originales equidistantes
print("\nCargando datos originales...")
with open('pkl/gam_original.pkl', 'rb') as f:
    df_original = pickle.load(f)

# Identificar especies
especies_cols = [col for col in df_original.columns if col != 'cal BP']
n_especies = len(especies_cols)
print(f"Especies: {n_especies}")

# ============================================
# PASO 2: MATRICES ORIGINALES POR VENTANA
# ============================================

print("\n" + "="*60)
print("PASO 2: CALCULANDO MATRICES ORIGINALES POR VENTANA")
print("="*60)

matrices_originales = {}

for window_name, (cal_min, cal_max) in windows.items():
    print(f"\n--- Ventana: {window_name} ---")
    
    # Extraer ventana
    df_window = extraer_ventana(df_original, cal_min, cal_max)
    print(f"  Puntos en ventana: {len(df_window)}")
    
    # Calcular matriz
    matriz = calcular_matriz_coeficientes(df_window, especies_cols, window_name)
    matrices_originales[window_name] = matriz
    
    # Estadísticas básicas
    n_pos = np.sum(matriz > 0)
    n_neg = np.sum(matriz < 0)
    print(f"  Coeficientes positivos: {n_pos}")
    print(f"  Coeficientes negativos: {n_neg}")

# Guardar matrices originales
with open('matrices_originales_ventanas.pkl', 'wb') as f:
    pickle.dump({
        'matrices': matrices_originales,
        'especies': especies_cols,
        'windows': windows
    }, f)

print(f"\n✅ Matrices originales guardadas en 'matrices_originales_ventanas.pkl'")

# ============================================
# PASO 3: MATRICES BOOTSTRAP POR VENTANA
# ============================================

print("\n" + "="*60)
print(f"PASO 3: CALCULANDO MATRICES BOOTSTRAP POR VENTANA")
print("="*60)

# Estructura para guardar: [ventana][bootstrap][i, j]
matrices_bootstrap = {window_name: [] for window_name in windows.keys()}

for b in range(1, N_BOOTSTRAP + 1):
    print(f"\n--- Bootstrap #{b} ---")
    
    # Cargar datos bootstrap
    filename = f'pkl/gam_boost_{b}.pkl'
    with open(filename, 'rb') as f:
        df_bootstrap = pickle.load(f)
    
    for window_name, (cal_min, cal_max) in windows.items():
        print(f"  Ventana: {window_name}")
        
        # Extraer ventana
        df_window = extraer_ventana(df_bootstrap, cal_min, cal_max)
        
        # Calcular matriz
        matriz = calcular_matriz_coeficientes(df_window, especies_cols, f"{window_name}_b{b}")
        matrices_bootstrap[window_name].append(matriz)

# Convertir a arrays 3D
for window_name in windows.keys():
    matrices_bootstrap[window_name] = np.array(matrices_bootstrap[window_name])

print(f"\nDimensiones:")
for window_name in windows.keys():
    print(f"  {window_name}: {matrices_bootstrap[window_name].shape}")

# Guardar matrices bootstrap
with open('matrices_bootstrap_ventanas.pkl', 'wb') as f:
    pickle.dump({
        'matrices': matrices_bootstrap,
        'especies': especies_cols,
        'windows': windows,
        'n_bootstrap': N_BOOTSTRAP
    }, f)

print(f"\n✅ Matrices bootstrap guardadas en 'matrices_bootstrap_ventanas.pkl'")

# ============================================
# PASO 4: ANÁLISIS DE SIGNIFICANCIA POR VENTANA
# ============================================

print("\n" + "="*60)
print("PASO 4: ANÁLISIS DE SIGNIFICANCIA POR VENTANA")
print("="*60)

resultados = {}

for window_name in windows.keys():
    print(f"\n--- Ventana: {window_name} ---")
    
    matriz_orig = matrices_originales[window_name]
    matrices_boost = matrices_bootstrap[window_name]
    n = n_especies
    
    # Inicializar matrices de resultados
    pct_2_5 = np.zeros((n, n))
    pct_97_5 = np.zeros((n, n))
    significativo = np.zeros((n, n), dtype=bool)
    cola = np.zeros((n, n), dtype='U12')  # 'bajo', 'alto', 'no'
    
    for i in range(n):
        for j in range(n):
            if i != j:
                # Distribución bootstrap para este par
                valores_boost = matrices_boost[:, i, j]
                valores_boost = valores_boost[~np.isnan(valores_boost)]
                
                if len(valores_boost) >= N_BOOTSTRAP * 0.5:  # Suficientes valores
                    # Calcular percentiles
                    pct_2_5[i, j] = np.percentile(valores_boost, 2.5)
                    pct_97_5[i, j] = np.percentile(valores_boost, 97.5)
                    
                    # Comparar con valor original
                    valor_orig = matriz_orig[i, j]
                    
                    if valor_orig < pct_2_5[i, j]:
                        significativo[i, j] = True
                        cola[i, j] = 'bajo'
                    elif valor_orig > pct_97_5[i, j]:
                        significativo[i, j] = True
                        cola[i, j] = 'alto'
                    else:
                        significativo[i, j] = False
                        cola[i, j] = 'no'
                else:
                    pct_2_5[i, j] = np.nan
                    pct_97_5[i, j] = np.nan
                    significativo[i, j] = False
                    cola[i, j] = 'insuficiente'
    
    # Guardar resultados para esta ventana
    resultados[window_name] = {
        'matriz_original': matriz_orig,
        'percentil_2_5': pct_2_5,
        'percentil_97_5': pct_97_5,
        'significativo': significativo,
        'cola': cola
    }
    
    # Estadísticas
    n_pares = n * (n - 1)
    n_sig = np.sum(significativo)
    n_bajos = np.sum(cola == 'bajo')
    n_altos = np.sum(cola == 'alto')
    
    print(f"\n  Resultados:")
    print(f"    Pares totales: {n_pares}")
    print(f"    Significativos: {n_sig} ({n_sig/n_pares*100:.2f}%)")
    print(f"      Cola baja (<2.5%): {n_bajos} ({n_bajos/n_pares*100:.2f}%)")
    print(f"      Cola alta (>97.5%): {n_altos} ({n_altos/n_pares*100:.2f}%)")

# ============================================
# PASO 5: GUARDAR RESULTADOS COMPLETOS
# ============================================

resultados_completos = {
    'especies': especies_cols,
    'windows': windows,
    'resultados_por_ventana': resultados,
    'n_bootstrap': N_BOOTSTRAP,
    'alpha': ALPHA
}

with open('resultados_granger_bootstrap_ventanas.pkl', 'wb') as f:
    pickle.dump(resultados_completos, f)

print(f"\n✅ Resultados completos guardados en 'resultados_granger_bootstrap_ventanas.pkl'")

# ============================================
# RESUMEN FINAL
# ============================================

print("\n" + "="*70)
print("RESUMEN FINAL - PORCENTAJES POR VENTANA")
print("="*70)

print("\nVentana        | Total Pares | Significativos | % | Cola baja | Cola alta")
print("-"*70)

for window_name in windows.keys():
    res = resultados[window_name]
    n = n_especies
    n_pares = n * (n - 1)
    n_sig = np.sum(res['significativo'])
    n_bajos = np.sum(res['cola'] == 'bajo')
    n_altos = np.sum(res['cola'] == 'alto')
    
    print(f"{window_name:12} | {n_pares:11} | {n_sig:13} | "
          f"{n_sig/n_pares*100:5.2f}% | {n_bajos:8} ({n_bajos/n_pares*100:4.1f}%) | "
          f"{n_altos:8} ({n_altos/n_pares*100:4.1f}%)")

print("\n" + "="*70)
print("✅ SCRIPT 2 COMPLETADO")
print("="*70)

SCRIPT 2: ANÁLISIS GRANGER CON BOOTSTRAP POR VENTANAS

Parámetros:
  Umbral p-value: 0.01
  Número bootstraps: 5
  Alpha (IC 95%): 0.05
  Ventanas: ['9798_6253', '6182_3842', '3771_-57']

PASO 1: CARGANDO DATOS

Cargando datos originales...
Especies: 75

PASO 2: CALCULANDO MATRICES ORIGINALES POR VENTANA

--- Ventana: 9798_6253 ---
    Ventana 6253-9798: 51 puntos
  Puntos en ventana: 51
  Calculando matriz 75x75 para 9798_6253...
    Progreso: 500/5550
    Progreso: 1000/5550


In [20]:
# ============================================
# DIAGNÓSTICO DE LA FUNCIÓN GRANGER
# ============================================

print("\n" + "="*70)
print("DIAGNÓSTICO DE LA FUNCIÓN GRANGER")
print("="*70)

# Cargar datos
with open('pkl/gam_original.pkl', 'rb') as f:
    df_original = pickle.load(f)

# Ventana de ejemplo
window_name = "9798_6253"
cal_min, cal_max = 9798, 6253
df_window = df_original[(df_original['cal BP'] >= cal_min) & (df_original['cal BP'] <= cal_max)].copy()

print(f"\nVentana {window_name}: {len(df_window)} puntos")

# Probar con diferentes pares
pares_prueba = [
    ('Pinus', 'Abies'),
    ('Pinus', 'Poaceae'),
    ('Poaceae', 'Artemisia'),
    ('Artemisia', 'Chenopodiaceae')
]

for sp1, sp2 in pares_prueba:
    if sp1 not in df_window.columns or sp2 not in df_window.columns:
        print(f"\n{sp1} → {sp2}: ❌ Especie no encontrada")
        continue
        
    print(f"\n{sp1} → {sp2}:")
    
    # Mostrar estadísticas básicas de las series
    print(f"  {sp1} - media: {df_window[sp1].mean():.4f}, std: {df_window[sp1].std():.4f}")
    print(f"  {sp2} - media: {df_window[sp2].mean():.4f}, std: {df_window[sp2].std():.4f}")
    
    # Probar cálculo manual
    try:
        array_x = df_window[sp1].values[::-1]
        array_y = df_window[sp2].values[::-1]
        
        print(f"  Primeros 5 valores (invertidos):")
        print(f"    {sp1}: {array_x[:5]}")
        print(f"    {sp2}: {array_y[:5]}")
        
        # Verificar si son constantes
        if np.std(array_x) < 1e-10:
            print(f"  ⚠️ {sp1} es casi constante (std={np.std(array_x):.6f})")
        if np.std(array_y) < 1e-10:
            print(f"  ⚠️ {sp2} es casi constante (std={np.std(array_y):.6f})")
        
        # Ruido
        ruido_x = np.random.normal(0, np.std(array_x) * 1e-6, len(array_x))
        ruido_y = np.random.normal(0, np.std(array_y) * 1e-6, len(array_y))
        
        array_x = array_x + ruido_x
        array_y = array_y + ruido_y
        
        # Estandarizar
        data = pd.DataFrame({sp1: array_x, sp2: array_y})
        scaler = StandardScaler()
        data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
        
        print(f"  Datos estandarizados - media: {data_scaled.mean().values}, std: {data_scaled.std().values}")
        
        # VAR
        model = VAR(data_scaled)
        results = model.fit(maxlags=1)
        
        print(f"  Coeficientes VAR:")
        print(f"    {sp1}: {results.coefs[0][0]}")
        print(f"    {sp2}: {results.coefs[0][1]}")
        
        coeficiente = results.coefs[0][1][0]
        print(f"  Coeficiente {sp1}→{sp2}: {coeficiente:.6f}")
        
        # Test de causalidad
        causality = results.test_causality(caused=sp2, causing=[sp1], kind='f')
        print(f"  p-value: {causality.pvalue:.6f}")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")


DIAGNÓSTICO DE LA FUNCIÓN GRANGER

Ventana 9798_6253: 0 puntos

Pinus → Abies:
  Pinus - media: nan, std: nan
  Abies - media: nan, std: nan
  Primeros 5 valores (invertidos):
    Pinus: []
    Abies: []
  ❌ Error: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.

Pinus → Poaceae:
  Pinus - media: nan, std: nan
  Poaceae - media: nan, std: nan
  Primeros 5 valores (invertidos):
    Pinus: []
    Poaceae: []
  ❌ Error: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.

Poaceae → Artemisia:
  Poaceae - media: nan, std: nan
  Artemisia - media: nan, std: nan
  Primeros 5 valores (invertidos):
    Poaceae: []
    Artemisia: []
  ❌ Error: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.

Artemisia → Chenopodiaceae:
  Artemisia - media: nan, std: nan
  Chenopodiaceae - media: nan, std: nan
  Primeros 5 valores (invertidos):
    Artemisia: []
    Chenopo